# Dynamic Spectral Downscaling

This is an example of dynamical downscaling, using hourly spectral data to feed SWAN model in Nonstationary mode. 

SWAN scenario is a demo case based on Roi-namur (Kwajalein atoll).


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# dev library
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# swan wrap module
from hywaves.swan.wrap import SwanProject, SwanMesh, SwanWrap_NONSTAT
from hywaves.swan.plots.stationary import scatter_maps


In [2]:
# --------------------------------------
# data
p_data = op.abspath(op.join(os.path.abspath(''), '..', '..', 'data'))
p_demo = op.join(p_data, 'demo', 'roi')

# swan main
p_swan_ext = op.join(p_demo, 'Roinamur_ext_bottom.dat')

# waves data
p_waves = op.join(p_data, 'demo', 'waves_efth_demo.nc')

# roi shore (for plots)
p_shore = op.join(p_demo, 'shore.npy')
np_shore = np.load(p_shore)


In [3]:
# load
efth = xr.open_dataset(p_waves)

# add time as "index" to efth dataset
efth['index'] = efth.time

# calculate U10, V10 from wspeed and wdir
efth['U10'] = efth.Wspeed * np.sin(np.deg2rad(efth.Wdir))
efth['V10'] = efth.Wspeed * np.cos(np.deg2rad(efth.Wdir))

## SWAN Project

In [4]:
# --------------------------------------
# SWAN project 

p_proj = op.join(p_data, 'projects')  # swan projects main directory
n_proj = 'nb_05_demo'                 # project name

sp = SwanProject(p_proj, n_proj)


### 1.1. SWAN: main grid description <a name="sp-grid-m"></a>

In [5]:
# --------------------------------------
# SWAN main mesh
main_mesh = SwanMesh()

# depth grid description (input bathymetry grid)
main_mesh.dg = {
    'xpc': 166.6159,  # x origin
    'ypc': 8.4105,    # y origin
    'alpc': 0,        # x-axis direction 
    'xlenc': 1.3722,  # grid length in x
    'ylenc': 1.175,   # grid length in y
    'mxc': 137,       # number mesh x
    'myc': 118,       # number mesh y
    'dxinp': 0.01,    # size mesh x
    'dyinp': 0.01,    # size mesh y
}

# depth value
main_mesh.depth = np.loadtxt(p_swan_ext)

# computational grid description
main_mesh.cg = main_mesh.dg.copy()

sp.set_main_mesh(main_mesh)


### 1.2. SWAN: case parameters <a name="sp-params"></a>

In [6]:
# --------------------------------------
# SWAN parameters (sea level, jonswap gamma...)

input_params = {
    'set_level': 0,
    'set_convention': 'NAUTICAL',

    'coords_mode': 'SPHERICAL',
    'coords_projection': 'CCM',
    
    'boundw_jonswap': 10,
    'boundw_period': 'PEAK',
    
    'output_deltt': '1 HR',
    'compute_deltc': '10 MIN',

    'boundn_mode': 'CLOSED',

    'physics':[
        'FRICTION JONSWAP',
        'BREAKING',
    ],

    'numerics':[
        'OFF QUAD',
        'PROP BSBT',
    ]
}

sp.set_params(input_params)


### 1.3. SWAN: Build and launch waves cases <a name="sp-cases"></a>

In [8]:
# SWAN project wrapper
sw = SwanWrap_NONSTAT(sp)

# build and launch cases
sw.build_spectral_cases(efth, make_winds=False)
sw.run_cases()


### 1.4. SWAN: main mesh output <a name="sp-output-mg"></a>

In [10]:
# extract output for main mesh
xds_out = sw.extract_output()
print(xds_out)


<xarray.Dataset>
Dimensions:     (case: 1, lat: 118, lon: 137, time: 2)
Coordinates:
  * time        (time) datetime64[ns] 2014-12-19 2014-12-19T01:00:00
  * lon         (lon) float64 166.6 166.6 166.6 166.6 ... 168.0 168.0 168.0
  * lat         (lat) float64 8.411 8.42 8.43 8.44 ... 9.546 9.556 9.566 9.576
  * case        (case) int64 0
Data variables:
    Depth       (case, time, lat, lon) float32 4.978e+03 4.984e+03 ... 4.574e+03
    Tm02        (case, time, lat, lon) float32 nan nan nan ... 2.032 2.033 2.034
    Tp          (case, time, lat, lon) float32 nan nan nan ... 1.998 1.999 1.999
    Hsig        (case, time, lat, lon) float32 0.0 0.0 ... 1.788e-07 1.772e-07
    RTpeak      (case, time, lat, lon) float32 nan nan nan ... 1.941 1.941 1.941
    Dspr        (case, time, lat, lon) float32 nan nan nan ... 7.576 1.72 0.1136
    FSpr        (case, time, lat, lon) float32 nan nan nan ... 0.8332 0.8341
    Dir         (case, time, lat, lon) float32 nan nan nan ... 262.4 262.5 262.5
  